# Processing of DIC data using numpy/scipy

Enable plotting using the `widget` front-end of matplotlib and import numpy and matplotlib packages as `np` and `plt`.

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pylab as plt
import pandas as pd

Import the regular grid interpolator

In [ ]:
from scipy.interpolate import LinearNDInterpolator
from scipy.spatial import Delaunay

In [ ]:
import os.path as op
import os

## Read the history of displacements and strains

In [ ]:
XU_TQa_file = '/home/data/S16.npz'
data = np.load(XU_TQa_file)

In [ ]:
X_TQa, U_TQa = data['X_TQa'], data['U_TQa']

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
T = 0
args = (X_TQa[T,:,0], X_TQa[T, :, 1], eps_TQ[T]) #  X_TQa[-1,:,2])
ax.scatter(*args);

### Displacement at nodes

In [ ]:
X_0Qa = X_TQa[0]
U_TQa =  X_TQa - X_0Qa[None, ...]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
T = 1
U_factor = 10
X_scaled_TQa = X_0Qa + U_factor * U_TQa
args = (X_scaled_TQa[T, :, 0], X_scaled_TQa[T, :, 1])
ax.scatter(*args);
ax.axis('equal');

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
T = -1
U_factor = 1
X_scaled_TQa = X_0Qa + U_factor * U_TQa
#args = (X_scaled_TQa[T, :, 0], X_scaled_TQa[T, :, 1], eps_TQ)
ax.plot_trisurf(*(X_TQa[T].T));

In [ ]:
points = X_0Qa[:, :-1]
delauney = Delaunay(points)

## Displacement interpolation grid to calculate strains

In [ ]:
pad_b, pad_t, pad_r, pad_l = 40, 40, 40, 40

In [ ]:
X_outer_frame = np.min(X_0Qa, axis=0), np.max(X_0Qa, axis=0)
X_outer_frame

In [ ]:
n_a = 2

### Interpolation grid

In [ ]:
X_min_a, X_max_a = X_outer_frame
L_a = X_max_a - X_min_a # frame dimensions
d = 4 # 4 mm distances
n_a = 2 # number of spatial dimensions
n_I, n_J = np.array( L_a[:-1] / d, dtype=np.int_ )
d_X_a = [L_a[0]/n_I, L_a[1]/n_J]
n_I, n_J

In [ ]:
min_x, min_y, _ = X_min_a
max_x, max_y, _ = X_max_a
X_aIJ = np.mgrid[
        min_x + pad_l:max_x - pad_r:complex(n_I),
        min_y + pad_b:max_y - pad_t:complex(n_J)]
x_IJ, y_IJ = X_aIJ
X0_IJa = np.einsum('aIJ->IJa', np.array([x_IJ, y_IJ]))

In [ ]:
x0_IJ, y0_IJ = np.einsum('IJa->aIJ', X0_IJa)
U_IJa_list = []
for T in range(n_T):
    values = U_TQa[T, :, :]
    get_U = LinearNDInterpolator(delauney, values)
    U_IJa = get_U(x0_IJ, y0_IJ)
    U_IJa_list.append(U_IJa)
U_TIJa = np.array(U_IJa_list)[...,:-1]

## Strain field history

In [ ]:
I,J,a = [np.arange(n) for n in (n_I,n_J,n_a)]
G_aIJ = (np.einsum('a, I->aI', (1-a), I)[:,:, None] + 
         np.einsum('a, J->aJ', a, J)[:,None, :])

In [ ]:
n_i, n_j = 2, 2
g_aij = G_aIJ[:,:n_i,:n_j]
n_E, n_F = n_I-1, n_J-1
G_aEF = G_aIJ[:,:-1,:-1]
H_aEiFj = G_aEF[:,:,None,:,None] + g_aij[:,None,:,None,:]
X_EiFja = X0_IJa[(*H_aEiFj,)]
U_TEiFja = U_TIJa[(slice(None), *H_aEiFj)]

In [ ]:
delta_rs = np.eye(2, dtype=np.int_)
xi_rij = (H_aEiFj[:,0,:,0,:] * 2) - 1
n_m, n_n = n_i, n_j
eta_rmn = 3**(-1/2) * xi_rij
N_ijmn = (1 + 
  np.einsum('rmn,rij->ijmn', eta_rmn, xi_rij) +
  np.einsum('rs,smn,sij,rmn,rij->ijmn', (1-delta_rs), eta_rmn, xi_rij, eta_rmn, xi_rij) / 2
)/4

In [ ]:
dN_sijmn = (
    xi_rij[:,:,:,None,None] + 
    np.einsum('rs,sij,rmn,rij->sijmn', (1 - delta_rs), xi_rij, eta_rmn, xi_rij)
) / 4
J_EmFnas = np.einsum(
 'sijmn,EiFja->EmFnas',
 dN_sijmn, X_EiFja
)
inv_J_EmFnsa = np.linalg.inv(J_EmFnas)

In [ ]:
delta_ab = np.eye(2)
Diff1_abcd = 0.5 * (
    np.einsum('ac,bd->abcd', delta_ab, delta_ab) +
    np.einsum('ad,bc->abcd', delta_ab, delta_ab)
)
B_EiFjmnabc = np.einsum(
    'abcd,sijmn,EmFnsd->EiFjmnabc',
    Diff1_abcd, dN_sijmn, inv_J_EmFnsa
)

In [ ]:
eps_TEmFnab = np.einsum(
    'EiFjmnabc,TEiFjc->TEmFnab',
    B_EiFjmnabc, U_TEiFja
)
eps_TEmFnab.shape

In [ ]:
eps_TEmFna, _ = np.linalg.eig(eps_TEmFnab)

In [ ]:
kappa_TEmFn = np.max(eps_TEmFna, axis=-1)
kappa_TKL = kappa_TEmFn.reshape(-1, n_E*n_m, n_F*n_n)
kappa_TKL[np.where(kappa_TKL<0)] = 0

In [ ]:
eps_0=0.002
#eps_f=0.0028
eps_f=0.003

In [ ]:
I = np.where(kappa_TEmFn>=eps_0)
omega_TEmFn = np.zeros_like(kappa_TEmFn)
omega_TEmFn[I] = 1.0-(eps_0/kappa_TEmFn[I]*np.exp(
    -(kappa_TEmFn[I]-eps_0)/(eps_f-eps_0))
)

In [ ]:
X_aEmFn = np.einsum('ijmn,EiFja->aEmFn', N_ijmn, X_EiFja)
X_aKL = X_aEmFn.reshape(-1,(n_I-1)*2, (n_J-1)*2)

In [ ]:
omega_TKL = omega_TEmFn.reshape(-1, n_E*n_m, n_F*n_n)

In [ ]:
fig = plt.figure(figsize=(12,5), tight_layout=True)
fig.canvas.header_visible = False
T_selection = [0, -100, -30, -1]
rows = fig.subplots(len(T_selection), 2)
for (ax_eps, ax_omega), T in zip(rows, T_selection):
    ax_eps.contourf(X_aKL[0], X_aKL[1], kappa_TKL[T], cmap='PuRd', 
                    levels=np.linspace(0,0.05,15)) # , vmin=eps_0, vmax=50*eps_0,)
    ax_eps.axis('equal');
    ax_eps.axis('off');
    ax_eps.set_title(r'$\varepsilon(T={}$)'.format(T));
    ax_omega.contourf( X_aKL[0], X_aKL[1], omega_TKL[T], cmap='BuPu', levels=np.linspace(0,1,15))
#                      vmin=0, vmax=1)
    ax_omega.axis('equal');
    ax_omega.axis('off');
    ax_omega.set_title(r'$\omega(T={})$'.format(T));

In [ ]:
eps_TKLa = eps_TEmFna.reshape(-1, n_E*n_m, n_F*n_n, n_a)

In [ ]:
np.max(eps_TKLa)

In [ ]:
eps_TKL_comp = np.copy(eps_TKLa[:, :, :, 1])
eps_TKL_comp[eps_TKL_comp>0] = 0
eps_TKL_comp[eps_TKLa[:, :, :, 0] > 0.005 ] = 0
eps_min_val = np.min(eps_TKL_comp)
eps_min_val

In [ ]:
fig = plt.figure(figsize=(8,5), tight_layout=True)
fig.canvas.header_visible = False
T = 20
ax = fig.subplots(1,1)
ax.contourf(X_aKL[0], X_aKL[1], eps_TKL_comp[-1], cmap='PuRd_r'), 
#                    levels=np.linspace(0,1,10)) # , vmin=eps_0, vmax=50*eps_0,)
ax.axis('equal');
ax.axis('off');
ax.set_title(r'$\varepsilon(T={})$'.format(T));

In [ ]:
eps_TKLab = eps_TEmFnab.reshape(-1, n_E*n_m, n_F*n_n, n_a, n_a)
sig_TKLab = np.einsum(',...,...ab->...ab', 30000, (1 - omega_TKL), eps_TKLab)

In [ ]:
E_TKL = np.einsum('...ab,...ab->...', sig_TKLab, eps_TKLab) 

In [ ]:
np.max(E_TKL), np.min(E_TKL)

In [ ]:
fig = plt.figure(figsize=(8,5), tight_layout=True)

T = 20
ax = fig.subplots(1,1)
ax.contourf(X_aKL[0], X_aKL[1], E_TKL[T], cmap='PuRd'), 
#                    levels=np.linspace(0,1,10)) # , vmin=eps_0, vmax=50*eps_0,)
ax.axis('equal');
ax.axis('off');
ax.set_title(r'$E(T={})$'.format(T));